<a href="https://colab.research.google.com/github/sethgis/12/blob/main/LANDCOVER_AREAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://githubtocolab.com/giswqs/geemap/blob/master/examples/notebooks/113_image_area.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [ ]:
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 33.0 MB/s 
     |████████████████████████████████| 3.0 MB 38.7 MB/s 
     |████████████████████████████████| 98 kB 8.2 MB/s 
     |████████████████████████████████| 1.2 MB 44.2 MB/s 
     |████████████████████████████████| 3.3 MB 48.6 MB/s 
     |████████████████████████████████| 46 kB 4.2 MB/s 
     |████████████████████████████████| 8.8 MB 50.2 MB/s 
     |████████████████████████████████| 130 kB 66.8 MB/s 
     |████████████████████████████████| 99 kB 9.1 MB/s 
     |████████████████████████████████| 1.3 MB 47.5 MB/s 
     |████████████████████████████████| 95 kB 3.7 MB/s 
     |████████████████████████████████| 187 kB 74.8 MB/s 
     |████████████████████████████████| 128 kB 7.8 MB/s 
     |████████████████████████████████| 54 kB 3.1 MB/s 
     |████████████████████████████████| 342 kB 49.5 MB/s 
     |████████████████████████████████| 42

In [ ]:
import ee
import geemap

Add ESA Land Cover data.

In [ ]:
Map = geemap.Map()
table = ee.FeatureCollection("users/snyawacha/LASWE_BOUNDARY_SYMMETRICAL_DIFFERENCE");
dataset = ee.ImageCollection("ESA/WorldCover/v100").first().clip(table)
Map.addLayer(dataset, {'bands': ['Map']}, 'ESA Land Cover')
Map.add_legend(builtin_legend='ESA_WorldCover')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
task2 = ee.batch.Export.image.toDrive(**{
    'image': dataset,
    'description': 'lulc_EA',
    'folder':'Erodible',
    'scale': 100,
    'region': table.geometry(),
    'maxPixels':1100000000,
    
})
task2.start()

Calculate the area of each land cover type.

In [ ]:
df = geemap.image_area_by_group(dataset, scale=1000, denominator=1e6, decimal_places=4, verbose=True)
df

Calculating area for group 10 ...
Calculating area for group 20 ...
Calculating area for group 30 ...
Calculating area for group 40 ...
Calculating area for group 50 ...
Calculating area for group 60 ...
Calculating area for group 80 ...
Calculating area for group 90 ...
Calculating area for group 95 ...


,area,percentage
group,,
10,2.992775e+05,0.0576
20,1.521451e+06,0.2926
30,1.453857e+06,0.2796
40,3.847317e+05,0.0740
50,4.353705e+03,0.0008
60,1.456682e+06,0.2802
80,1.271372e+04,0.0024
90,6.572842e+04,0.0126
95,6.418385e+02,0.0001


Save the results to a CSV.

In [ ]:
CSV = df.to_csv('ea_lulc_area.csv')
print(CSV)


None


Add NLCD land cover data.

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)
Map.add_basemap('HYBRID')

nlcd = ee.Image('USGS/NLCD_RELEASES/2019_REL/NLCD/2019')
landcover = nlcd.select('landcover')
Map.centerObject(table)
Map.addLayer(landcover, {}, 'NLCD Land Cover 2019')
Map.add_legend(title="NLCD Land Cover Classification", builtin_legend='NLCD', height='465px')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

Calculate the area of each land cover type.

In [ ]:
df = geemap.image_area_by_group(landcover, scale=1000, denominator=1e6, decimal_places=4, verbose=True)
df

Save the results to a CSV.

In [ ]:
df.to_csv('nlcd_area.csv')